First, we set up a function that generates $N$ cities randomly in a $1\times1$ square box and one that calculates the distance between each two of them. The input is the number of cities $N$ and the city coordinates, and the output is city coordinates and an $N\times N$ matrix with its $i,j$ element being the distance between city $i$ and $j$ respectively.

In [17]:
import numpy as np
import random

def generate_cities(N):
    # Generate N cities randomly in a 1x1 square box
    cities = np.random.rand(N, 2)  # Each city is represented by its (x, y) coordinates
    return cities

def calculate_distances(cities, N):
    distances = np.zeros((N, N))  # Initialize an N x N matrix for distances
    
    for i in range(N):
        for j in range(N):
            distances[i][j] = np.sqrt(np.sum((cities[i] - cities[j]) ** 2))  # Calculate Euclidean distance
    
    return distances

The next function generates an initial guess for the traveling salesman's shortest route. Starting from city $i=0$, find its nearest neighbour $j$; then find $j$'s nearest neighbour except for $i$, let's say it's $k$; then find $k$'s nearest neighbour $l$ except for $i$ and $j$; keep doing this until each of the $N$ cities has appeared once. This function takes the distance matrix and the city number as input and gives an $N$-element sequence recording the route as output.

In [2]:
def generate_initial_route(distances, N):
    route = [0]  # Start from city 0
    visited = set([0])  # Keep track of visited cities
    
    while len(route) < N:
        # Find the nearest neighbour of the last city in the route that hasn't been visited yet
        min_dist = float('inf')  # Initialize to a large value
        next_city = -1  # Index of the next city to add to the route
        for j in range(N):
            if j not in visited:  # Ignore already visited cities
                dist = distances[route[-1]][j]  # Distance from the last city in the route to city j
                if dist < min_dist:  # Update the minimum distance and next city
                    min_dist = dist
                    next_city = j
        route.append(next_city)  # Add the next city to the route
        visited.add(next_city)  # Mark the city as visited
    
    return route  # The initial route as a sequence of city indices

The minimal change of a route is a swap between two city routes (route[$(i-1)$%$N$],route[$i$]) and (route[$j$],route[$(j+1)$%$N$]), $i < j < N$, which trades the original city routes for (route[$(i-1)$%$N$],route[$j$]) and (route[$i$],route[$(j+1)$%$N$]), which is to reverse the city sequence from route[$i$] to route[$j$]. The swap makes sense only when the length of the reversed sequence is no longer than $N-2$, i.e. $j - i + 1 <= N - 2$ or $j - i < N - 2$. Now let's define the swap function with their input being a route and the swap indices $i$ and $j$ and their output being the swapped route or the warning information of illegal swap indices $i$ and $j$.

In [5]:
def swap_route_segment(route, N, i, j):
    # Check if the swap is within bounds and makes sense
    if i < j and j - i + 1 <= N - 2:  # Condition for valid swap
        # Perform the swap by reversing the sequence from route[i] to route[j]
        route[i:j+1] = reversed(route[i:j+1])  # Swap the segment
        return route  # Return the swapped route
    else:
        return f"Warning: Illegal swap indices {i} and {j}. Swap not performed."

Now we want a function that decides whether or not to swap. Define the energy $E$ as the length difference between the swapped and unswapped route, i.e. $E$ = distances[route[$(i-1)$%$N$]][route[$j$]] + distances[route[$i$]][route[$(j+1)$%$N$]] - distances[route[$(i-1)$%$N$]][route[$i$]] - distances[route[$j$]][route[$(j+1)$%$N$]]. If $E < 0$, we always want to do the swap, and the function should return True. If $E > 0$, we swap with a $e^{-E/T}$ probability, where $T$ is a temperature from the input. This function takes the distances matrix, the route, the city number, the swap indices $i$ and $j$, and the temperature $T$ as input, and returns True or False, i.e. whether or not to swap.

In [ ]:
def should_swap(distances, route, N, i, j, T):
    # Calculate the energy difference between the swapped and unswapped route
    E = distances[route[(i-1) % N][route[j]] + distances[route[i]][route[(j+1) % N]] - distances[route[(i-1) % N][route[i]] - distances[route[j]][route[(j+1) % N]]
    
    # If E is negative, always accept the swap
    if E < 0:
        return True
    
    # If E is positive, accept the swap with probability e^{-E/T}
    if np.exp(- E / T) > np.random.rand():  # Using NumPy's random number generator for speed and efficiency
        return True
    else:
        return False

Now we define a function that calculates the standard deviation of all the city routes and then triples it as the initial temperature. Use the distance matrix and city number as input, and return the initial temperature. Notice that the matrix diagonal is not wanted in the calculation of the standard deviation.

In [1]:
def calculate_initial_temperature(distances, N):
    # Calculate the mean value of squared distances
    squared_distances = np.square(distances)
    sum_squared_distances = np.sum(squared_distances)
    mean_squared_distances = sum_squared_distances / (N * (N - 1))
    
    # Calculate the mean value of distances
    sum_distances = np.sum(distances)
    mean_distances = sum_distances / (N * (N - 1))
    
    # Calculate the standard deviation of all city routes
    std_dev = np.sqrt(mean_squared_distances - mean_squared_distances ** 2)
    
    # Triple the standard deviation as the initial temperature
    initial_temperature = 3 * std_dev
    
    return initial_temperature

With $N$ being the number of cities, there are also $N$ city routes in any given route, and therefore $N (N - 1) / 2$ possible ways of swap. For the cooling-down strategy, we reduce the temperature by $0.1 *$ initial_temperature every $N (N - 1) / 2$ steps, which are taken as a measure of the time elapsed. Here comes the temperature function that takes initial_temperature, current step number, and city number, and returns the current temperature.

In [ ]:
def current_temperature(initial_temperature, step, N):
    # Calculate the number of possible swaps (N*(N-1)/2)
    num_swaps = N * (N - 1) // 2  # Using integer division
    
    # Determine the current step's temperature decrement
    temperature_decrement = 0.1 * initial_temperature  # Decrease by 10% of initial temperature
    
    # Calculate the current temperature based on the step number and decrement
    current_temperature = initial_temperature - (step // num_swaps) * temperature_decrement
    
    # Ensure the temperature doesn't fall below a minimum threshold (e.g., a small positive value)
    current_temperature = max(current_temperature, temperature_decrement)  # Adjust this minimum threshold as needed
    
    return current_temperature

Here is a function that calculates the route length given the distances matrix and the route.

In [ ]:
def calculate_route_length(distances, route, N):
    route_length = 0  # Initialize the length of the route to zero
    for i in range(N):
        # Calculate the distance between the current city and the next one in the route
        city_i = route[i]
        city_j = route[(i + 1) % N]  # Use modulo to loop back to the start if necessary
        route_length += distances[city_i][city_j]  # Add the distance to the total route length
    return route_length

Now comes the function that searches for the shortest route. Given the distances matrix, city number, initial temperature, current route, current step number, and current rotation index, the search function goes through every possible swap $i,j$, checks by should_swap(), and records the step number. If False, do nothing except add 1 to the step number; if True, do the swap $i,j$, rotate the route to the left by $i$ to make city route[$i$] the first element in the route, add to the rotation index by $i$, add 1 to the step and then return with the swapped route, current step number, updated rotation index and a True. If all possible swaps are checked with none triggered, return with the current route, step number, rotation index, and a False.

In [ ]:
def search_for_shortest_route(distances, N, initial_temperature, route, step, rotation_index):
    found = False # Flag to indicate if any swaps were made during this iteration
    # Check all possible swaps
    for i in range(N):
        for j in range(i + 1, N):  # Only check swaps with a different city index to avoid duplicates
            T = current_temperature(initial_temperature, step, N) # Calculate the temperature
            if should_swap(distances, route, N, i, j, T):  # Call your custom function to determine if a swap should be made
                route = swap_route_segment(route, N, i, j) # Make the swap
                route = route[i:] + route[:i] # Do the rotation
                rotation_index = rotation_index + i # Record the rotation index
                step = step + 1 # Record the step
                found = True # Found one
                return route, step, rotation_index, found
            else:
                step = step + 1 # Record the step
    return route, step, rotation_index, found